# Import 


In [15]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np 
import pandas  as pd
import datetime as dt
import openpyxl
from scipy.stats import skew, kurtosis, linregress
from scipy.signal import welch, find_peaks
from scipy.stats import mannwhitneyu
from sklearn.metrics import roc_curve, auc
import scipy.stats as stats

# Récuperation des patients  

In [16]:
df_codesPatients = pd.read_excel(
        fr"C:\Users\bryan\fluid-response-analysis\data\codesPatients.xlsx"
    )
df_codesPatients

,Code Patient,Repondeur / non repondeur,dan
0,AA00,NaN,ok
1,AB01,responder,ok
2,AC02,responder,ok
3,AD03,non responder,ok
4,AD04,responder,ok
5,AE05,non responder,ok
6,AE06,responder,ok
7,AE07,non responder,ok
8,AF08,responder,ok
9,BB01,responder,ok


# I - fonctions niveau 1

In [17]:
# recuperation des segments de flux de chaque patient 

def get_segments(time_values, event_markers, flux):
    """
    time_values : liste de float (temps en secondes)
    event_markers : liste de [datetime.time, label]
    flux : liste de float
    """

    # --- Conversion des entrées ---
    time_values = np.asarray(time_values, dtype=float)
    flux = np.asarray(flux, dtype=float)

    # --- Extraction + conversion des markers en secondes ---
    event_markers_seconds = np.asarray(
        [
            t
            for t, _ in event_markers
        ],
        dtype=float
    )


    # --- Recherche des indices de découpage ---
    time_position = []

    for t in event_markers_seconds:
        pos = np.where(time_values >= t)[0]
        if len(pos) > 0:
            time_position.append(int(pos[0]))
    
    if len(time_position) != 0 and time_position[0] != 0:
        time_position = [0]+[ i for i in time_position]

    # --- Ajouter la fin du signal ---
    if len(time_position) != 0 and time_position[-1] != len(time_values):
        time_position.append(len(time_values))

    # --- Découpage des segments ---
    segments = []
    for i in range(len(time_position) - 1):
        segments.append(flux[time_position[i]:time_position[i + 1]])

    return segments


# recuperation des segments de flux de chaque patient 
def get_segments_0(t, time,flux):
    #index des time dans t
    time_position= []
    for i in time : 
        position = t.index(i)
        time_position.append(position)

    #ajouter le dernier le dernier indice a la liste des times positions 
    time_position.append(len(t))  # je ne met pas len -1 comme ca dans la formation des segments il vas s'arreter a stop -1 ie len -1

    #obtention des differents segment de données 
    segments =  []
    for i in range(len(time_position)-1):
        segments.append(flux[time_position[i]:time_position[i+1]])
    
    return segments

def to_seconds(t):
    if pd.isna(t):
        return None
    parts = str(t).split(":")
    parts = [float(p.replace(",", ".")) for p in parts]
    
    if len(parts) == 3:      # HH:MM:SS
        h,m,s = parts
        return h*3600 + m*60 + s
    elif len(parts) == 2:    # MM:SS
        m,s = parts
        return m*60 + s
    elif len(parts) == 1:    # SS
        return parts[0]
    else:
        return None


def times_markers_areas_in_seconds(df):
    """
    Extrait tous les 'Time' des MARKERS + les limites des AREAS
    et renvoie une liste de temps en secondes.

    """
    
    #-------- 1) repérer MARKERS et AREAS dans le fichier -------

    # Trouver ligne "MARKERS"
    idx_markers = df[df.iloc[:,0] == "MARKERS"].index[0] + 1
    
    # trouver ligne "AREAS"
    idx_areas = df[df.iloc[:,0] == "AREAS"].index[0] + 1

    # -------- 2) Extraire MARKERS (Times seulement) -------
    df_markers = df.iloc[idx_markers:idx_areas]      # section markers
    df_markers = df_markers.iloc[1:].reset_index(drop=True)
    
    marker_times = df_markers[["Unnamed: 1","Unnamed: 3"]].dropna().values.tolist()

    # -------- 3) conversion en secondes -------
    if marker_times :
        for marker in marker_times:
            if marker[0] is not None:
                marker[0] = to_seconds(marker[0])
    else :
        marker_times = []

    return marker_times

def get_usefull_data(df):

    """
    Extrait tous les données entre des MARKERS + les limites des AREAS
    et renvoie une liste de temps en secondes.
    
    """
    
    #-------- 1) repérer RAW DATA et GENERAL CALCULATIONS : AREA 1 dans le fichier -------
    # Trouver ligne "MARKERS"
    idx_raw_data = df[df.iloc[:,0] == "RAW DATA"].index[0] + 1
    
    # trouver ligne "AREAS"
    idx_area_1 = df[df.iloc[:,0] == "GENERAL CALCULATIONS : AREA 1"].index[0] + 0

    # -------- 2) Extraire MARKERS (Times seulement) -------
    df_data = df.iloc[idx_raw_data:idx_area_1]      # section markers

    first_row_list = df_data.iloc[0].tolist()

    df_data.columns = first_row_list

    df_data = df_data.iloc[1:].reset_index(drop=True)

    #supprime toute les lignes ou Sample vaut <Segement break> ou Sample 
    df_data = df_data[
        ~df_data["Sample"].isin(["<Segment break>", "Sample"])
    ].reset_index(drop=True)

    return df_data

def basic_stats(x: np.ndarray, name,statut) -> dict:
    """
    calcul les statistique de base (les metriques )
    
    """
    x = np.asarray(x).astype(float)
    n = len(x)

    # Axe temporel
    fs = 1
    t = np.arange(n) / fs

    # Pente (tendance temporelle)
    slope = linregress(t, x).slope

    # Statistiques de base
    mean_val = np.mean(x)
    min_val = np.min(x)
    max_val = np.max(x)

    # Amplitude pic–pic
    amp_pp = max_val - min_val

    # Indice de pulsatilité
    pulsatility_index = amp_pp / mean_val if mean_val != 0 else np.nan

    # Aire sous la courbe
    auc = np.trapezoid(x, t)

    return {
        "name": name,
        "statut" : statut,
        "mean": np.mean(x),
        "median": np.median(x),
        "min": np.min(x),
        "max": np.max(x),
        "amplitude": np.ptp(x),          # max - min
        "var": np.var(x, ddof=1),
        "std": np.std(x, ddof=1),
        "q25": np.percentile(x, 25),
        "q75": np.percentile(x, 75),
        "skew": skew(x, bias=False),
        "kurtosis": kurtosis(x, bias=False, fisher=True),
        "rms": np.sqrt(np.mean(x**2)),
        "trend_slope": slope,
        "pulsatility_index": pulsatility_index,
        "auc": auc,
       
    }

def spectral_features(x: np.ndarray, fs: float) -> dict:
    """
        calcul les metriques d'analyse frequentielle 
    
    """
    x = np.asarray(x).astype(float)

    # PSD par Welch (robuste pour du physiologique)
    f, Pxx = welch(x, fs=fs, nperseg=min(256, len(x)))
    total_power = np.trapz(Pxx, f)

    # exemples de bandes (à adapter à ton signal) :
    def band_power(fmin, fmax):
        mask = (f >= fmin) & (f <= fmax)
        return np.trapz(Pxx[mask], f[mask]) if np.any(mask) else 0.0

    low = band_power(0.04, 0.15)   # ex. bande "LF" (HRV)
    high = band_power(0.15, 0.4)   # ex. bande "HF"
    lf_hf = (low / high) if high > 0 else np.nan

    # fréquence dominante
    dominant_idx = np.argmax(Pxx)
    f_dom = f[dominant_idx] if Pxx.size else np.nan

    # entropie spectrale (Shannon) sur PSD normalisée
    P = Pxx / (np.sum(Pxx) + 1e-12)
    spec_entropy = -np.sum(P * np.log2(P + 1e-12))

    return {
        "total_power": float(total_power),
        "band_power_low": float(low),
        "band_power_high": float(high),
        "lf_hf_ratio": float(lf_hf),
        "f_dominant_hz": float(f_dom),
        "spectral_entropy": float(spec_entropy),
    }


def basic_stats_for_segment(special_times, all_times):
    """
    special_times : valeurs-temps repères (doivent exister dans all_times)
    all_times     : liste/array des temps
    Retourne une liste de stats par segment [t_i, t_{i+1})
    """
    # 1) récupérer les indices des repères dans all_times
    special_index = []
    for t in special_times:
        try:
            idx = all_times.index(t)   # .index() lève ValueError si non trouvé
            special_index.append(idx)
        except ValueError:
            print("erreur de recuperation de l'index :", t)

    # Besoin d'au moins 2 repères pour former 1 segment
    if len(special_index) < 2:
        return []

    # 2) calcul des stats par segment
    statistics = []
    for i in range(len(special_index) - 1):
        start, end = special_index[i], special_index[i + 1]
        # slice Python : all_times[start:end] (end exclu)
        x = np.array(all_times[start:end])
        stat = basic_stats(x,"zzz","non reponder")   # suppose que basic_stats(x) existe
        statistics.append(stat)

    return statistics

def spectrale_features_for_segment(special_times, all_times):
    """
    special_times : valeurs-temps repères (doivent exister dans all_times)
    all_times     : liste/array des temps
    Retourne une liste de spectrale feature par segment [t_i, t_{i+1})
    """
    # 1) récupérer les indices des repères dans all_times
    special_index = []
    for t in special_times:
        try:
            idx = all_times.index(t)   # .index() lève ValueError si non trouvé
            special_index.append(idx)
        except ValueError:
            print("erreur de recuperation de l'index :", t)

    # Besoin d'au moins 2 repères pour former 1 segment
    if len(special_index) < 2:
        return []

    # 2) calcul des stats par segment
    statistics = []
    for i in range(len(special_index) - 1):
        start, end = special_index[i], special_index[i + 1]
        # slice Python : all_times[start:end] (end exclu)
        x = np.array(all_times[start:end])
        stat = spectral_features(x,1)   # suppose que basic_stats(x) existe
        statistics.append(stat)

    return statistics




# II - fonctions niveau 2

In [18]:
def displaywithspetialtimes(patient):
    """
    Affiche l'évolution du flux (colonne '3 PU') pour un patient donné, en alignant le
    signal temporel sur le premier point mesuré. Ajoute également des lignes verticales
    marquant les temps d'événements (markers) extraits du fichier source.

    Paramètres
    ----------
    patient : str
        Identifiant du patient (nom du fichier sans extension).
    """
    # --- Chargement du fichier Excel du patient ---
    dataframe_raw = pd.read_excel(
        fr"C:\Users\bryan\fluid-response-analysis\data\{patient}.xls"
    )
    
    # --- Extraction des données utiles ---
    dataframe_clean = get_usefull_data(dataframe_raw)
    
    # --- Récupération des marqueurs temporels (en secondes) ---
    event_markers_seconds = times_markers_areas_in_seconds(dataframe_raw)

    # Conversion en secondes
    dataframe_clean["time_seconds"] = dataframe_clean["Time"].apply(
        lambda t: (
            t.hour * 3600 +
            t.minute * 60 +
            t.second +
            t.microsecond / 1e6
        ) if isinstance(t, dt.time) else None
    )

    # --- Temps de référence (t0 = premier timestamp) ---
    reference_time = dataframe_clean.loc[0, "time_seconds"]

    if reference_time is not None:

        # --- Extraction des données temps + flux ---
        flux_values = []
        time_values = []

        for index, row in dataframe_clean.iterrows():
            flux_values.append(row["3 PU"])
            time_values.append(row["time_seconds"])

        # --- Recentrage des marqueurs et du signal temporel ---
        for marker in event_markers_seconds:
            marker[0] = marker[0]-reference_time 
        
        time_values = [t - reference_time for t in time_values]

        # --- Création de la figure ---
        fig = go.Figure()
        fig.update_layout(title=f"Évolution du flux du patient {patient}")
        fig.update_xaxes(title_text="Temps (s)")
        fig.update_yaxes(title_text="Amplitude (mV)")

        # --- Ajout de la courbe de flux ---
        fig.add_trace(go.Scatter(x=time_values, y=flux_values, name="Flux"))

        # --- Ajout des lignes verticales correspondant aux marqueurs ---
        for i, marker in enumerate(event_markers_seconds):
            fig.add_vline(
                x=marker[0],
                line_color="red", 
                annotation_text=f"T{i + 1} = {marker[1]}"
            )

        # --- Affichage ---
        fig.show()

def get_time_and_flux_values_both_center_in_0(patient):
    """
    recupère le temps et les flux
    Paramètres
    ----------
    patient : str
        Identifiant du patient (nom du fichier sans extension).
    columns : list
        Liste des colonnes à extraire du fichier.
    """
    # --- Chargement du fichier Excel du patient ---
    dataframe_raw = pd.read_excel(
        fr"C:\Users\bryan\fluid-response-analysis\data\{patient}.xls"
    )
    
    # --- Extraction des données utiles ---
    dataframe_clean = get_usefull_data(dataframe_raw)

    # Conversion en secondes
    dataframe_clean["time_seconds"] = dataframe_clean["Time"].apply(
        lambda t: (
            t.hour * 3600 +
            t.minute * 60 +
            t.second +
            t.microsecond / 1e6
        ) if isinstance(t, dt.time) else None
    )

    time_values = []
    flux_values = []

    for _, row in dataframe_clean.iterrows():
        time_values.append(row["time_seconds"])
        try:
            flux_values.append(float(row["3 PU"]))
        except (TypeError, ValueError):
            continue
    
    # --- Temps et flux de référence les indices 0 ---
    reference_time = time_values[0]
    reference_flux = flux_values[0]

    time_values = [t - reference_time for t in time_values]
    flux_values = [f - reference_flux for f in flux_values]

    
    print("flux = ", flux_values)
    print("time = ",time_values)
    return time_values, flux_values

def display():
    """
    Affiche l'évolution temporelle du flux sanguin pour l'ensemble des patients,
    en séparant les patients répondeurs et non répondeurs.

    Deux figures sont générées :
    - une pour les patients répondeurs
    - une pour les patients non répondeurs
    """
# --- Création de la figure des repondeur ---
    fig_reponder = go.Figure()
    fig_reponder.update_layout(title=f"Évolution du flux des patients repondeurs ")
    fig_reponder.update_xaxes(title_text="Temps (s)")
    fig_reponder.update_yaxes(title_text="Amplitude (mV)")

# --- Création de la figure des non repondeur ---
    fig_non_reponder = go.Figure()
    fig_non_reponder.update_layout(title=f"Évolution du flux des patients non repondeurs")
    fig_non_reponder.update_xaxes(title_text="Temps (s)")
    fig_non_reponder.update_yaxes(title_text="Amplitude (mV)")
    patients_en_erreur = []
    for _,row in df_codesPatients.iterrows():
        patient = row["Code Patient"]
        try :
            print("========debut ",patient,"=========")
            time_values, flux_values = get_time_and_flux_values_both_center_in_0(patient)
            # --- Ajout de la courbe de flux ---
            if row["Repondeur / non repondeur"] and row["Repondeur / non repondeur"] == "responder" :           
                fig_reponder.add_trace(go.Scatter(x=time_values, y=flux_values, name=patient))
            elif row["Repondeur / non repondeur"] and row["Repondeur / non repondeur"] == "non responder" :
                fig_non_reponder.add_trace(go.Scatter(x=time_values, y=flux_values, name=patient))
        except Exception as e:
            patients_en_erreur.append(patient)
            print("========",patient,"=========")
            print(f"Erreur pour {patient} : {e}")
    # --- Affichage ---
    fig_non_reponder.show()
    # --- Affichage ---
    fig_reponder.show()

    print(patients_en_erreur)

def get_time_and_flux_values(patient):
    """
    Affiche l'évolution du flux (colonne '3 PU') pour un patient donné, en alignant le
    signal temporel sur le premier point mesuré. Ajoute également des lignes verticales
    marquant les temps d'événements (markers) extraits du fichier source.

    Paramètres
    ----------
    patient : str
        Identifiant du patient (nom du fichier sans extension).

    """
    # --- Chargement du fichier Excel du patient ---
    dataframe_raw = pd.read_excel(
        fr"C:\Users\bryan\fluid-response-analysis\data\{patient}.xls"
    )
    
    # --- Extraction des données utiles ---
    dataframe_clean = get_usefull_data(dataframe_raw)
    
    # --- Récupération des marqueurs temporels (en secondes) ---
    event_markers_seconds = times_markers_areas_in_seconds(dataframe_raw)

    # Conversion en secondes
    dataframe_clean["time_seconds"] = dataframe_clean["Time"].apply(
        lambda t: (
            t.hour * 3600 +
            t.minute * 60 +
            t.second +
            t.microsecond / 1e6
        ) if isinstance(t, dt.time) else None
    )

    # --- Temps de référence (t0 = premier timestamp) ---
    reference_time = dataframe_clean.loc[0, "time_seconds"]

    if reference_time is not None:

        # --- Extraction des données temps + flux ---
        flux_values = []
        time_values = []

        for index, row in dataframe_clean.iterrows():
            flux_values.append(row["3 PU"])
            time_values.append(row["time_seconds"])

        # --- Recentrage des marqueurs et du signal temporel ---
        for marker in event_markers_seconds:
            marker[0] = marker[0]-reference_time
            marker[0] = float(marker[0]) 
        
        time_values = [t - reference_time for t in time_values]

        return event_markers_seconds,time_values,flux_values

def basic_stats_patient(patient,statut):
    """
    Calcule les statistiques de base pour un patient donné.

    Les statistiques sont calculées :
    - pour chaque segment temporel du signal
    - puis pour le signal complet (sans segmentation)

    Paramètres
    ----------
    patient : str
        Identifiant du patient
    statut : str
        Statut du patient (répondeur / non répondeur)

    Retour
    ------
    DataFrame
        Tableau contenant les statistiques calculées
        pour chaque segment et pour le signal complet
    """

    event_markers_seconds,time_values,flux_values = get_time_and_flux_values(patient)
    segments = get_segments(time_values,event_markers_seconds,flux_values)

    # calcul des statistique de base du patient MR 
    stats = []
    for index,x in enumerate(segments) :
        result = basic_stats(x,f"{patient}_segment_{index+1}",statut)   
        stats.append(result)
    
    #stats sans découpage 
    result = basic_stats(flux_values,f"{patient}_tout",statut) 
    stats.append(result)
    

    df_basic_stats = pd.DataFrame(stats)

    return df_basic_stats

# III - recupération de toute les métriques 


In [19]:
def metriquesparsegment(segment):
    """
    Extrait les métriques statistiques correspondant à un segment donné
    pour l'ensemble des patients.

    Paramètre
    ----------
    segment : int
        Numéro du segment à analyser (indexé à partir de 1)

    Retour
    ------
    DataFrame
        Table contenant les métriques du segment sélectionné
        pour tous les patients valides
    """

    # Liste pour stocker les statistiques de tous les patients
    all_segments_stats = []

    # Liste pour garder la trace des patients ayant généré une erreur
    patients_en_erreur = []

    # Boucle sur tous les patients référencés
    for _, patient in df_codesPatients.iterrows():

        # Récupération des informations patient
        code_patient = patient["Code Patient"]
        statut_patient = patient["Repondeur / non repondeur"]

        try:
            # Calcul des statistiques pour le patient
            df = basic_stats_patient(code_patient, statut_patient)

            # Si aucune donnée n'est disponible, on passe au patient suivant
            if df.empty:
                continue

            # Conversion du numéro de segment (1-based) en index Python (0-based)
            i = segment - 1

            # Vérifie que le segment demandé existe et qu'il ne contient pas uniquement des NaN
            if len(df) > i and not df.iloc[i].isna().all():
                # Ajout des métriques du segment à la liste globale
                all_segments_stats.append(df.iloc[i])

        except Exception as e:
            # En cas d'erreur, on stocke le patient concerné
            patients_en_erreur.append(code_patient)
            print("========", code_patient, "=========")
            print(f"Erreur pour {code_patient} : {e}")

    # Création du DataFrame final à partir des segments valides
    df_all_segments_stats = pd.DataFrame(all_segments_stats).reset_index(drop=True)

    # Suppression du patient UI03_tout : son signal de flux est beaucoup plus élevé que les autres et est considéré comme du bruit
    df_all_segments_stats = df_all_segments_stats[
        df_all_segments_stats['name'] != "UI03_tout"
    ]

    # Suppression du patient AC02_tout :  dans ce cas, il ne possède pas de segmentation exploitable
    if i != -1:
        df_all_segments_stats = df_all_segments_stats[
            df_all_segments_stats['name'] != "AC02_tout"
        ]

    # Retourne les métriques du segment sélectionné pour tous les patients
    return df_all_segments_stats
